<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/Group_3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with PyTorch
## 3D Convolutional Neural Network on Group Brain fMRI
Contributors: Ben Merrill, Stacey Rivet Beck

This CNN is implemented in similar form to the fMRI CNN dicussed in the paper by [Wang et al.](https://arxiv.org/pdf/1801.09858.pdf)

You can see the architecture of the CNN in the class ConvNet below. Throughout the model training, we used cross-entropy loss and back propogation to update, keeping track of both accuracy and loss at each epoch. Due to RAM limitations, we trained the model from 4 files. Each file was split into two sets of 756 images and used for training. The first run through each of the 8 datasets entailed 10 epochs. The second run-through contained one epoch for each dataset, to avoid overfitting the model.

Once the training was complete, the model measured error and trained one epoch on the validation set. Finally, the model runs the test data and returns the predictions and the error. 

Although this script only shows the training portion of the model building, it could very easily be updated for model validation and testing.

## Importing Dataset and Labels

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git

# Change directory into cloned repo DL folder
%cd teambrainiac/source/DL

# !ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 1859, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 1859 (delta 31), reused 25 (delta 25), pack-reused 1815
Receiving objects: 100% (1859/1859), 100.81 MiB | 12.49 MiB/s, done.
Resolving deltas: 100% (1222/1222), done.
/content/teambrainiac/source/DL


### Load path_config.py

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 196 bytes


## Import Packages

In [30]:
# # Possible Missing Packages
# !pip install boto3
# !pip install nilearn

In [5]:
# General Library Imports
import re
import scipy.io
import os
import pickle
import numpy as np
import nibabel as nib
import pandas as pd
import boto3
import tempfile
import tqdm
import random
from path_config import mat_path
from botocore.exceptions import ClientError
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

# From Local Directory
from access_data_dl import *
from process_dl import *

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import TensorDataset

#import torchvision.transforms as transforms
from torch.nn import ReLU, CrossEntropyLoss, Conv3d, Module, Softmax, AdaptiveAvgPool3d
from torch.optim import Adam, SGD

#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader


## Import Group fMRI Data from AWS

In [20]:
%%time
path = 'dl/partition_train_2.pkl'
train_images = access_load_data(path, False)

CPU times: user 32.3 s, sys: 1min 3s, total: 1min 35s
Wall time: 1min 38s


In [21]:
train_images['images'].shape

torch.Size([1512, 1, 79, 95, 79])

## Build Model

In [24]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    #Conv1
    self.conv1 = nn.Conv3d(in_channels = 1, 
                           out_channels = 32, 
                           kernel_size = (1,1,1), 
                           stride = (1,1,1)
                           )
    self.bn1 = nn.BatchNorm3d(32)
    self.conv2 = nn.Conv3d(in_channels = 32, 
                           out_channels = 64, 
                           kernel_size = (7,7,7),
                           stride = (2,2,2)
                           )
    self.bn2 = nn.BatchNorm3d(64)
    self.conv3 = nn.Conv3d(in_channels = 64, 
                           out_channels = 64, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn3 = nn.BatchNorm3d(64)
    self.conv4 = nn.Conv3d(in_channels = 64, 
                           out_channels = 128, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn4 = nn.BatchNorm3d(128) 
    self.pool1 = nn.AdaptiveAvgPool3d((1,1,1)) #Global Average Pool, takes the average over last two dimensions to flatten 
  
                                                             
    # Fully connected layer
    self.fc1 = nn.Linear(128,64) # need to find out the size where AdaptiveAvgPool 
    self.fc2 = nn.Linear(64, 2) # left with 2 for the two classes                     

  def forward(self, xb):
    xb = self.bn1(F.relu(self.conv1((xb))))
    xb = self.bn2(F.relu(self.conv2((xb)))) # Takes a long time
    xb = self.bn3(F.relu(self.conv3((xb))))
    xb = self.bn4(F.relu(self.conv4((xb))))
    xb = self.pool1(xb)
    xb = xb.view(xb.shape[:2])
    xb = self.fc1(xb)
    xb = self.fc2(xb)
    return xb      

    

## Functions to Run the Model

In [26]:

def run_cnn(model, epochs, learning_rate, loss_func, opt, dl):
  metrics_dict = {}
  # Run Model
  for epoch in range(1, 1+epochs):
    i = 1
    accuracy_list = []
    loss_list = []
    model.train()
    print('epoch', epoch)
    for xb, yb in dl:
      print('batch', i)
      i += 1

      xb = xb.float()
      pred = model(xb)
      loss_batch = loss_func(pred, yb)
      loss_list.append(loss_batch)
      accuracy_batch = accuracy(pred, yb)
      if int(accuracy_batch) == 1:
        print('Perfect Accuracy\nStopping early to avoid overfitting\n\n')
        break


      accuracy_list.append(accuracy_batch)

      print('Batch Loss', loss_batch)
      print('Batch Accuracy', accuracy_batch)

      loss_batch.backward()
      opt.step()
      opt.zero_grad()

    model.eval()

    metrics_dict['epoch_'+str(epoch)] = {'accuracy':accuracy_list, 'loss':loss_list}

    print('epoch', epoch, 'finished\n')
    try:
      past_epoch_accuracies = [sum(metrics_dict['epoch_'+str(epoch-2)]['accuracy']), sum(metrics_dict['epoch_'+str(epoch-1)]['accuracy'])]
      current_epoch_accuracy = sum(metrics_dict['epoch_'+str(epoch)]['accuracy'])
      if past_epoch_accuracies[0] > current_epoch_accuracy and past_epoch_accuracies[1] > current_epoch_accuracy:
        print('Early stop to avoid overfitting\nModel accuracies did not decrease for two epochs')
        return model, metrics_dict

    except:
      pass
  
  return model, metrics_dict
  

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()



## Setting Up and Running the Model

In [31]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
# print("First model training on GPU")
# print(model)

# Initialize other parameters
epochs = 2
learning_rate = 0.0005
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)



In [32]:
# Load older models for continued training
load_recent_model = True

if load_recent_model:
  path = '/content/gdrive/My Drive/cnn_train_2_round_2.pt'
  model.load_state_dict(torch.load(path))

In [34]:

%%time

# Pick the dataset to train on (1, 2, 3, or 4)
train_index = 3

# Load the training data
path = 'dl/partition_train_%i.pkl'%(train_index)
train_images = access_load_data(path, False)

CPU times: user 29.6 s, sys: 41.8 s, total: 1min 11s
Wall time: 1min 26s


In [ ]:
# Run the model

metrics_dict = {}
for i,image_index in enumerate(range(0, train_images['images'].shape[0], 756)):

  bs = 54

  x_train = train_images['images'][image_index:image_index+756]
  y_train = train_images['labels'][image_index:image_index+756]

  ds = TensorDataset(x_train, y_train)
  dl = DataLoader(ds, batch_size = bs, shuffle=True)
  epochs = 1

  model, metrics = run_cnn(model, epochs, learning_rate, loss_func, opt, dl)
  metrics_dict['round_'+str(i)] = metrics
  
  f = open("/content/gdrive/My Drive/metrics_dict_train_4%s.pkl"%(str(i)),"wb")
  pickle.dump(metrics_dict,f)

  print('Saving model')
  model_name = 'cnn_fmri_initial_model.pt'
  path = F"/content/gdrive/My Drive/{model_name}" 
  torch.save(model.state_dict(), path)

  print('Finshed with set', str(i), 'of 504 images\nStarting next set.\n\n')

epoch 1
batch 1
Batch Loss tensor(0.4207, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.7963)


## Training

In [20]:
# infile = open('/content/gdrive/My Drive/metrics_dict_train_20.pkl','rb')
# best_model2 = pickle.load(infile)

In [19]:
# for x in best_model2['round_0']['epoch_6']['accuracy']:
#   print(int(x))